In [1]:
# %pip install -r requirements.txt

In [ ]:
import src._1_models as pmmd
import importlib

importlib.reload(pmmd)


file_path = "test_jira_reference_postman.json"

collection = pmmd.PostmanCollection.from_file(file_path, debug_prn=False)


In [3]:
# next((fd for fd in collection.get_subfolders() if 'Time' in fd.name))

collection.get_subfolders()

[PostmanFolder(name='Application Properties', items=[PostmanRequest(name='Get Application Property', method='GET', headers=[], url=PostmanUrl(raw='{{baseUrl}}/rest/api/3/application-properties?key=<string>&permissionLevel=<string>&keyFilter=<string>', protocol='https', host=['{{baseUrl}}'], path=['rest', 'api', '3', 'application-properties'], query=[PostmanQueryParam(key='key', value='<string>', description='The key of the application property.', disabled=False), PostmanQueryParam(key='permissionLevel', value='<string>', description='The permission level of all items being returned in the list.', disabled=False), PostmanQueryParam(key='keyFilter', value='<string>', description="When a `key` isn't provided, this filters the list of results by the application property `key` using a regular expression. For example, using `jira.lf.*` will return all application properties with keys that start with *jira.lf.*.", disabled=False)], variable=None), description='Returns all application properti

In [4]:
import src._2_converter as pmcv

importlib.reload(pmcv)

from pprint import pprint

config = {
    "drop_n_from_path_head": 3,
    "prefix": "v",
    "base_url_variable": "baseUrl",
    "signature_params": ["key", "keyFilter"],
}

cvrq = pmcv.PostmanRequestConverter.from_postman_request(
    collection.items[0].items[0], config=config
)


cvrq.export_code(
    config=config,
    export_base_folder="./EXPORT",
    is_add_imports=True,
    replace_folder=True,
    debug_prn=True,
)

Exporting to ./EXPORT\v3_application_properties_get.py


In [5]:
import src.utils.files as fi

config = {
    "drop_n_from_path_head": 3,
    "prefix": "v",
    "base_url_variable": "baseUrl",
    #   "signature_params" : ['key', 'keyFilter']
}

base_folder = "./EXPORT"

fi.upsert_folder(base_folder, replace_folder=True)

errata = []
for req in collection.get_folder_requests():
    try:
        cvrq = pmcv.PostmanRequestConverter.from_postman_request(req, config=config)

        cvrq.export_code(
            config=config,
            export_base_folder="./EXPORT",
            is_add_imports=True,
            replace_folder=False,
        )
    except AttributeError as e:
        errata.append(req)
        print(req._raw)
        print(f"Error processing request {req.name}: {e}")

        # raise e from e